In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable   

sys.path.append("../Preprocessor")
import format_module

import rnn
import naivebayesian

importing Jupyter notebook from rnn.ipynb


In [3]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [4]:
input_size = 100  # word2vec k size
batch_size = 10

In [5]:
class classifierModule(nn.Module):
    def __init__(self):
        super(classifierModule, self).__init__()
        self.rnn_model = rnn.RNN_model(input_size)
        
        self.nb_model = naivebayesian.NaiveBayesianDB()
        #TODO
        
    def encoder(self, formattedList):
        length = len(formattedList)
        contextList = [formattedList[i].context for i in range(length)]

        lengths = torch.LongTensor([len(contextList[i]) for i in range(length)])
        max_len = torch.max(lengths)
        
        data = np.zeros((length, max_len, input_size))

        for i in range(length):
            context = contextList[i]
            if not (context.size == 0):
                data[i, :context.shape[0],:] = context
            else:
                lengths[i] = 1
            i+=1
            
        return self.sort_batch(torch.FloatTensor(data), formattedList, lengths)
        
    def sort_batch(self, context, formatted, seq_len):
        batch_size = context.size(0)
        sorted_seq_len, sorted_idx = seq_len.sort(0, descending = True)
        
        sorted_context = context[sorted_idx]
        sorted_formatted = [formatted[i] for i in sorted_idx]

        for f in sorted_formatted:
            print(len(f.context))
        
        return Variable(sorted_context), sorted_formatted, sorted_seq_len
    
    def resize_input(self, input):
        list_ = list()
        for i in range(0, len(input), batch_size):
            list_.append(input[i:i+batch_size])
        return list_
        
    def forward(self, formatted_list, hidden=None):
        context, formatted, lengths = self.encoder(bl)
        
        print(self.rnn_model(context, lengths))
        self.nb_model.naive_bayes_FRlist(formatted)

In [7]:
formatted_list = load_object("../Preprocessor/save_formatted_review.pkl")
test_classifier = classifierModule()

test_classifier.nb_model.add_FRlist(formatted_list)

batch_list = test_classifier.resize_input(formatted_list)
for bl in batch_list:
    test_classifier(bl)

for param in test_classifier.parameters():
     print(type(param.data), param.size())

137
120
52
42
34
32
27
25
20
9
Variable containing:
 0.0424  0.0017 -0.0005 -0.1110 -0.0789 -0.1283  0.0683  0.3316 -0.0707 -0.0331
 0.0500 -0.0129 -0.0122 -0.1034 -0.1012 -0.1576  0.0671  0.3592 -0.0847 -0.0408
 0.0423 -0.0216 -0.0013 -0.1034 -0.0985 -0.1413  0.0588  0.3512 -0.0702 -0.0477
 0.0392 -0.0084 -0.0292 -0.1154 -0.1240 -0.1373  0.0695  0.3633 -0.0967 -0.0507
 0.0162  0.0086 -0.0039 -0.1015 -0.0975 -0.1300  0.0643  0.3492 -0.0805 -0.0432
 0.0217 -0.0092 -0.0124 -0.1111 -0.1155 -0.1357  0.0437  0.3523 -0.0628 -0.0476
 0.0441 -0.0013  0.0036 -0.1084 -0.0750 -0.1597  0.0634  0.3446 -0.0807 -0.0296
 0.0308 -0.0167 -0.0239 -0.0983 -0.1318 -0.1451  0.0415  0.3690 -0.0910 -0.0569
 0.0418 -0.0023 -0.0006 -0.1072 -0.1042 -0.1334  0.0526  0.3461 -0.0762 -0.0459
 0.0484 -0.0129 -0.0090 -0.0947 -0.1127 -0.1498  0.0464  0.3608 -0.0632 -0.0479
[torch.FloatTensor of size 10x10]



ZeroDivisionError: float division by zero

In [37]:
a = ["ㄱ", "ㄴ", "ㄷ", "ㄹ", "ㅁ"]
b = [4, 3, 2, 1, 0]
c = [4, 2, 1, 0, 3]

d = [x for _,x in sorted(zip(c,a))]
print(d)
list.reverse(d)
print(d)

['ㄹ', 'ㄷ', 'ㄴ', 'ㅁ', 'ㄱ']
['ㄱ', 'ㅁ', 'ㄴ', 'ㄷ', 'ㄹ']


In [10]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a_ = list()
for i in range(0, len(a), 3):
    a_.append(a[i:i+3])
print(a_)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [10]]
